In [1]:
import os
import sys

In [2]:
os.environ["PYSPARK_PYTHON"]="C:\Spark\spark-3.3.2-bin-hadoop2\python"
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-18.0.2.1"
os.environ["SPARK_HOME"] = "C:\Spark\spark-3.3.2-bin-hadoop2"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.9.5-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip") 

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("r_eda").getOrCreate()
spark

In [4]:
data = spark.read.option("header","true").csv("../dataset/r_hosp_demo_dataset.csv")
data.printSchema()

root
 |-- subject_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dod: string (nullable = true)
 |-- hadm_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- so2: string (nullable = true)
 |-- po2: string (nullable = true)
 |-- pco2: string (nullable = true)
 |-- fio2: string (nullable = true)
 |-- aado2: string (nullable = true)
 |-- ph: string (nullable = true)
 |-- baseexcess: string (nullable = true)
 |-- bg2_bicarbonate: string (nullable = true)
 |-- totalco2: string (nullable = true)
 |-- bg2_hematocrit: string (nullable = true)
 |-- bg2_haemoglobin: string (nullable = true)
 |-- carboxyhemoglobin: string (nullable = true)
 |-- methemoglobin: string (nullable = true)
 |-- bg2_chloride: string (nullable = true)
 |-- bg2_calcium: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- bg2_potassium: string (nullable = true)
 |-- bg2_sodium: string (nullable = true)
 |-- bg2_lactate: string (nullable = true)
 |-- bg2_glucose:

In [5]:
data.count()

275

In [6]:
data.describe().toPandas()

,summary,subject_id,gender,dod,hadm_id,age,so2,po2,pco2,fio2,...,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,crp,charlson_comorbidity_index
0,count,275,275,103,275,275,59,97,97,31,...,19,147,22,20,66,68,2,98,17,275
1,mean,1.0018901541818181E7,None,None,2.513054658909091E7,63.24551506545073,89.55932203389831,215.17525773195877,49.175257731958766,62.25806451612903,...,80.47368421052632,1.6231292517006808,3.236363636363636,2.3899999999999997,753.1515151515151,19.941176470588236,119.5,804.3877551020408,119.21764705882352,4.658181818181818
2,stddev,12127.768989002532,None,None,2907533.4283764022,14.498736578535334,15.296237069879457,139.02742612440406,14.354159029469436,22.908466763674202,...,86.75980150907871,3.7482976597641318,5.573623721178294,3.2701038129344484,2483.911656068209,73.4281824688914,37.476659402887016,2050.951916945327,101.68431685426052,2.744516158619529
3,min,10000032,F,2111-11-15,20044587,21.15688799024573661664,37,100,112,100,...,110,0.1,0.1,0.1,102,1,146,1040,0.4,0
4,max,10040025,M,2201-12-24,29974575,93.8402972093388311,99,98,94,94,...,96,9.5,6.9,5.9,99,9,93,995,87.8,9


- All the rows has subject, gender and admission id (hadm_id).
- The min and max age values looks good.

To Do
- Check for valid gender values and remove invalid genders (those other than 'M' and 'F').
- Remove columns with more than 50% missing values
- impute missing values by calculating the average of the value between the last & next reading for the same subject.
- Remove columns with more than 25% missing values after impuatation.
- remove rows which has data in less than 25% of the columns.

In [7]:
#checking for gender values
data.groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  133|
|     M|  142|
+------+-----+



Gender column values looks good.

In [8]:
# checking for columns with data in < 5% of the rows
dt = data.describe().toPandas()
dt

,summary,subject_id,gender,dod,hadm_id,age,so2,po2,pco2,fio2,...,amylase,bilirubin_total,bilirubin_direct,bilirubin_indirect,ck_cpk,ck_mb,ggt,ld_ldh,crp,charlson_comorbidity_index
0,count,275,275,103,275,275,59,97,97,31,...,19,147,22,20,66,68,2,98,17,275
1,mean,1.0018901541818181E7,None,None,2.513054658909091E7,63.24551506545073,89.55932203389831,215.17525773195877,49.175257731958766,62.25806451612903,...,80.47368421052632,1.6231292517006808,3.236363636363636,2.3899999999999997,753.1515151515151,19.941176470588236,119.5,804.3877551020408,119.21764705882352,4.658181818181818
2,stddev,12127.768989002532,None,None,2907533.4283764022,14.498736578535334,15.296237069879457,139.02742612440406,14.354159029469436,22.908466763674202,...,86.75980150907871,3.7482976597641318,5.573623721178294,3.2701038129344484,2483.911656068209,73.4281824688914,37.476659402887016,2050.951916945327,101.68431685426052,2.744516158619529
3,min,10000032,F,2111-11-15,20044587,21.15688799024573661664,37,100,112,100,...,110,0.1,0.1,0.1,102,1,146,1040,0.4,0
4,max,10040025,M,2201-12-24,29974575,93.8402972093388311,99,98,94,94,...,96,9.5,6.9,5.9,99,9,93,995,87.8,9


In [10]:
# identify and filter columns with < 50% missing values
dt_t = dt.T
dt_t.columns = dt_t.iloc[0]
dt_t.drop(dt_t.index[0], inplace=True)
#dt_t.drop(dt_t.index[89], inplace=True)
dt_t["count"] = dt_t['count'].astype(int)
dt_t['missing_percentage'] = 100 - (dt_t["count"] / data.count())*100
print(dt_t)
dt_t = dt_t[dt_t["missing_percentage"] < 50]
print(dt_t)
print(dt_t.shape)

summary                     count                  mean              stddev  \
subject_id                    275  1.0018901541818181E7  12127.768989002532   
gender                        275                  None                None   
dod                           103                  None                None   
hadm_id                       275   2.513054658909091E7  2907533.4283764022   
age                           275     63.24551506545073  14.498736578535334   
...                           ...                   ...                 ...   
ck_mb                          68    19.941176470588236    73.4281824688914   
ggt                             2                 119.5  37.476659402887016   
ld_ldh                         98     804.3877551020408   2050.951916945327   
crp                            17    119.21764705882352  101.68431685426052   
charlson_comorbidity_index    275     4.658181818181818   2.744516158619529   

summary                                         min

After removing all the columns with missing value % >= 50, we get remaining 45 feature fields which are as follows.

In [11]:
print(dt_t.shape)
dt_t.index

(45, 6)


Index(['subject_id', 'gender', 'hadm_id', 'age', 'db_wbc', 'basophils_abs',
       'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs',
       'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes',
       'monocytes', 'neutrophils', 'aniongap', 'bicarbonate', 'bun', 'calcium',
       'chloride', 'creatinine', 'glucose', 'sodium', 'potassium', 'inr', 'pt',
       'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet',
       'rbc', 'rdw', 'wbc', 'scr_min', 'ckd', 'mdrd_est', 'scr_baseline',
       'alt', 'alp', 'ast', 'bilirubin_total', 'charlson_comorbidity_index'],
      dtype='object')

In [12]:
# Filter out only the identified column data from the data
data = data.select(dt_t.index.values.tolist())

In [13]:
# round off age by 2 decimal point
print(data.printSchema())
data.describe().toPandas()

root
 |-- subject_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hadm_id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- db_wbc: string (nullable = true)
 |-- basophils_abs: string (nullable = true)
 |-- eosinophils_abs: string (nullable = true)
 |-- lymphocytes_abs: string (nullable = true)
 |-- monocytes_abs: string (nullable = true)
 |-- neutrophils_abs: string (nullable = true)
 |-- basophils: string (nullable = true)
 |-- eosinophils: string (nullable = true)
 |-- lymphocytes: string (nullable = true)
 |-- monocytes: string (nullable = true)
 |-- neutrophils: string (nullable = true)
 |-- aniongap: string (nullable = true)
 |-- bicarbonate: string (nullable = true)
 |-- bun: string (nullable = true)
 |-- calcium: string (nullable = true)
 |-- chloride: string (nullable = true)
 |-- creatinine: string (nullable = true)
 |-- glucose: string (nullable = true)
 |-- sodium: string (nullable = true)
 |-- potassium: string (nullable = true)
 |--

,summary,subject_id,gender,hadm_id,age,db_wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,...,wbc,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,charlson_comorbidity_index
0,count,275,275,275,275,245,141,141,141,141,...,245,250,275,275,271,146,144,147,147,275
1,mean,1.0018901541818181E7,None,2.513054658909091E7,63.24551506545073,16.230204081632642,0.186704255319149,0.4209907801418439,3.520324113475176,1.8931780141843977,...,16.230204081632642,1.0892,0.26181818181818184,0.9483440261915842,1.0588143183485186,157.4931506849315,128.5,255.0612244897959,1.6231292517006808,4.658181818181818
2,stddev,12127.768989002532,None,2907533.4283764022,14.498736578535334,17.86018748751892,0.7303802693384494,0.9996315214618342,6.808578087120277,5.817096760032841,...,17.86018748751892,0.9665506923922074,0.4404256853769406,0.12797510680700927,0.9260105173178504,719.6650777206676,85.63755140001497,1216.1321869870426,3.7482976597641318,2.744516158619529
3,min,10000032,F,20044587,21.15688799024573661664,0.7,0.0000,0.0000,0.2540,0.0700,...,0.7,0.1,0,0.76304145092510801528,0.1,100,100,1015,0.1,0
4,max,10040025,M,29974575,93.8402972093388311,98,6.3600,9.5000,9.7200,9.2460,...,98,6,1,1.22005455512109229427,6,98,99,98,9.5,9


Except for Gender, all other columns contain decimal values. Hence converting every column type to decimal with 2 decimal values.

In [54]:
# converting all number columns to double of precision 2 except for gender
import pyspark.sql.functions as F

cols = data.columns
cols.remove("gender")
data2 = data.select(*(F.round(F.col(c).cast("double"), 2).alias(c) for c in cols), "gender")
print(data2.columns)
print(data2.printSchema())
data2.describe().toPandas()

['subject_id', 'hadm_id', 'age', 'db_wbc', 'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs', 'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes', 'monocytes', 'neutrophils', 'aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc', 'scr_min', 'ckd', 'mdrd_est', 'scr_baseline', 'alt', 'alp', 'ast', 'bilirubin_total', 'charlson_comorbidity_index', 'gender']
root
 |-- subject_id: double (nullable = true)
 |-- hadm_id: double (nullable = true)
 |-- age: double (nullable = true)
 |-- db_wbc: double (nullable = true)
 |-- basophils_abs: double (nullable = true)
 |-- eosinophils_abs: double (nullable = true)
 |-- lymphocytes_abs: double (nullable = true)
 |-- monocytes_abs: double (nullable = true)
 |-- neutrophils_abs: double (nullable = true)
 |-- basophils: double (nullable = true)
 |-- eosinophils: double (null

,summary,subject_id,hadm_id,age,db_wbc,basophils_abs,eosinophils_abs,lymphocytes_abs,monocytes_abs,neutrophils_abs,...,scr_min,ckd,mdrd_est,scr_baseline,alt,alp,ast,bilirubin_total,charlson_comorbidity_index,gender
0,count,275,275,275,245,141,141,141,141,141,...,250,275,275,271,146,144,147,147,275,275
1,mean,1.0018901541818181E7,2.513054658909091E7,63.245490909090904,16.230204081632642,0.18652482269503545,0.42106382978723406,3.520425531914892,1.8931914893617021,13.137588652482266,...,1.0892,0.26181818181818184,0.94869090909091,1.0588929889298897,157.4931506849315,128.5,255.0612244897959,1.6231292517006808,4.658181818181818,None
2,stddev,12127.768989002532,2907533.4283764022,14.498681894371888,17.86018748751892,0.7304235423130522,0.9995368228246375,6.8085801826330465,5.817153491325577,14.132572250994672,...,0.9665506923922074,0.4404256853769406,0.1281383935819178,0.9260038954735257,719.6650777206676,85.63755140001497,1216.1321869870426,3.7482976597641318,2.744516158619529,None
3,min,1.0000032E7,2.0044587E7,21.16,0.7,0.0,0.0,0.25,0.07,0.21,...,0.1,0.0,0.76,0.1,7.0,28.0,5.0,0.1,0.0,F
4,max,1.0040025E7,2.9974575E7,93.84,182.6,6.36,9.5,57.41,58.43,97.52,...,6.0,1.0,1.22,6.0,8010.0,590.0,13060.0,35.9,12.0,M


In [ ]:
data2.filter((data2.subject_id == 10040025)).select("basophils_abs", "db_wbc", "platelet", "ast").show()

In [ ]:
# Before impute
data2.toPandas().to_csv("../dataset/EDA/before_imputation.csv")

In [64]:
# Use a window function
# https://sqlrelease.com/get-the-first-non-null-value-per-group-spark-dataframe
from pyspark.sql.window import Window
subject_win_prev = Window.partitionBy("subject_id").orderBy(F.desc("age")).rowsBetween(Window.currentRow+1,Window.unboundedFollowing)
subject_win_next = Window.partitionBy("subject_id").orderBy("age").rowsBetween(Window.currentRow+1,Window.unboundedFollowing)
#wi_next = Window.partitionBy("subject_id").orderBy(F.desc("age"))
for c in data2.columns:
    if ('missing_' not in c) and ('imputed_value' not in c) and (c not in ("subject_id", "age", "gender", "hadm_id", "charlson_comorbidity_index")):
            data2 = data2.withColumn('temp_' + c + 'prev', F.first(c, ignorenulls = True).over(subject_win_prev)) \
            .withColumn('temp_' + c + 'next', F.first(c, ignorenulls = True).over(subject_win_next)) \
            .withColumn(c , F.when(F.col(c).isNotNull() ,F.col(c)) \
                        .when(F.col(c).isNull() & F.col('temp_' + c + 'prev').isNull(), F.col('temp_' + c + 'next')) \
                        .when(F.col(c).isNull() & F.col('temp_' + c + 'next').isNull(), F.col('temp_' + c + 'prev')) \
                        .otherwise(((F.col('temp_' + c + 'prev') + F.col('temp_' + c + 'next'))/2))) \
            .drop('temp_' + c + 'prev', 'temp_' + c + 'next')

In [69]:
data2.filter((data2.subject_id == 10040025)).select("basophils_abs", "db_wbc", "platelet", "ast").show()

+-------------+------+--------+----+
|basophils_abs|db_wbc|platelet| ast|
+-------------+------+--------+----+
|         0.01|   8.9|   202.0|16.0|
|         0.01|   8.9|   238.5|16.0|
|         0.01|   8.9|   275.0|16.0|
|         0.01|   6.2|   258.0|16.0|
|         0.01|  12.6|   281.0|16.0|
|         0.01|  11.3|   307.0|16.0|
|         0.06|   9.8|   275.0|16.0|
|         0.06|   8.6|   323.0|16.0|
|         0.06|  12.6|   366.0|16.0|
|         0.11|  16.7|   349.0|16.0|
+-------------+------+--------+----+



In [ ]:
data2.toPandas().to_csv("../dataset/EDA/after_imputation.csv")